<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
from __future__ import division # / 运算执行的是精确除法
from __future__ import print_function

import time
import tensorflow as tf

import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import sys

In [3]:
def parse_index_file(filename):
    """Parse index file. """
    index = []
    for line in open(filename):
        index.append(int(line.strip())) # 读取文件每一行的数字， stirp() 移除前后的空格
    return index


def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)


def load_data(dataset_str):
    """
    Loads input data from gcn/data directory

    ind.dataset_str.x => the feature vectors of the training instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.tx => the feature vectors of the test instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.allx => the feature vectors of both labeled and unlabeled training instances
        (a superset of ind.dataset_str.x) as scipy.sparse.csr.csr_matrix object; (除去测试集的所有数据)
    ind.dataset_str.y => the one-hot labels of the labeled training instances as numpy.ndarray object;
    ind.dataset_str.ty => the one-hot labels of the test instances as numpy.ndarray object;
    ind.dataset_str.ally => the labels for instances in ind.dataset_str.allx as numpy.ndarray object;
    ind.dataset_str.graph => a dict in the format {index: [index_of_neighbor_nodes]} as collections.defaultdict
        object;
    ind.dataset_str.test.index => the indices of test instances in graph, for the inductive setting as list object.

    All objects above must be saved using python pickle module.

    :param dataset_str: Dataset name
    :return: All data input files loaded (as well the training/test data).
    """
    names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
    objects = []
    for i in range(len(names)):
        with open("data/ind.{}.{}".format(dataset_str, names[i]), 'rb') as f:
            if sys.version_info > (3, 0): # python 版本是否高于 3.0
                objects.append(pkl.load(f, encoding='latin1'))
            else:
                objects.append(pkl.load(f))

    x, y, tx, ty, allx, ally, graph = tuple(objects)
    test_idx_reorder = parse_index_file("data/ind.{}.test.index".format(dataset_str)) # 测试集数据的 index
    test_idx_range = np.sort(test_idx_reorder)

    if dataset_str == 'citeseer':
        # Fix citeseer dataset (there are some isolated nodes in the graph)
        # Find isolated nodes, add them as zero-vecs into the right position
        test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
        tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
        tx_extended[test_idx_range-min(test_idx_range), :] = tx
        tx = tx_extended
        ty_extended = np.zeros((len(test_idx_range_full), y.shape[1]))
        ty_extended[test_idx_range-min(test_idx_range), :] = ty
        ty = ty_extended

    features = sp.vstack((allx, tx)).tolil() # 先将数据堆叠，获得完整特征数据集， 之后转换为 lil 链表稀疏矩阵， 加速矩阵的访问。
    features[test_idx_reorder, :] = features[test_idx_range, :] # 调整测试集部分特征的顺序， 按照顺序进行排列
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph)) # 构建邻接矩阵

    labels = np.vstack((ally, ty)) # 堆叠获得完整的标签
    labels[test_idx_reorder, :] = labels[test_idx_range, :] # 调整测试集部分标签的顺序， 按照顺序进行排列

    idx_test = test_idx_range.tolist()
    idx_train = range(len(y))
    idx_val = range(len(y), len(y)+500)

    train_mask = sample_mask(idx_train, labels.shape[0])
    val_mask = sample_mask(idx_val, labels.shape[0])
    test_mask = sample_mask(idx_test, labels.shape[0])

    y_train = np.zeros(labels.shape)
    y_val = np.zeros(labels.shape)
    y_test = np.zeros(labels.shape)
    y_train[train_mask, :] = labels[train_mask, :] # 训练集标签
    y_val[val_mask, :] = labels[val_mask, :]
    y_test[test_mask, :] = labels[test_mask, :]

    return adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask

def sparse_to_tuple(sparse_mx):
    """Convert sparse matrix to tuple representation."""
    def to_tuple(mx):
        if not sp.isspmatrix_coo(mx):
            mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose() # 获得稀疏矩阵坐标 (2708, 1433)  --> (49216, 2)
        values = mx.data # 相应位置的值 (49216, ) 即矩阵中的所有非零值
        shape = mx.shape  # 稀疏矩阵的大小
        return coords, values, shape

    if isinstance(sparse_mx, list):
        # 如果是包含多个 matrix 的列表， 逐一转换
        for i in range(len(sparse_mx)):
            sparse_mx[i] = to_tuple(sparse_mx[i])
    else:
        sparse_mx = to_tuple(sparse_mx)

    return sparse_mx


def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    rowsum = np.array(features.sum(1)) # 每一行加和 ， (2708,1443) --> (2708,1)
    r_inv = np.power(rowsum, -1).flatten() # 求每个值的倒数 (2708,1) --> (2708,1)
    r_inv[np.isinf(r_inv)] = 0. # 将无穷大的值 isinf， 转化为 0
    r_mat_inv = sp.diags(r_inv) # 转换为对角矩阵 (2708,1) --> (2708,2708)
    features = r_mat_inv.dot(features) # 和原始矩阵相乘
    return sparse_to_tuple(features) # 转换为 tuple： 


def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix.  D^{-0.5}* A* D^{-0.5}"""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1)) # 每行求和 ， 
    d_inv_sqrt = np.power(rowsum, -0.5).flatten() # 和值求 ^{-0.5}
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0. #  inf 值赋 0 
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt) # 得到的 D 转换成为对角矩阵
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo() #  D^{-0.5}* A* D^{-0.5}


def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0])) # 加入 self-loop 之后重新归一化
    return sparse_to_tuple(adj_normalized)


def construct_feed_dict(features, support, labels, labels_mask, placeholders):
    """Construct feed dictionary."""
    feed_dict = dict()
    feed_dict.update({placeholders['labels']: labels}) # 更新字典的键值
    feed_dict.update({placeholders['labels_mask']: labels_mask})
    feed_dict.update({placeholders['features']: features}) # 
    feed_dict.update({placeholders['support'][i]: support[i] for i in range(len(support))}) # 多个图
    feed_dict.update({placeholders['num_features_nonzero']: features[1].shape}) # 非零特征值 数目
    return feed_dict


def chebyshev_polynomials(adj, k):
    """Calculate Chebyshev polynomials up to order k. Return a list of sparse matrices (tuple representation)."""
    print("Calculating Chebyshev polynomials up to order {}...".format(k))

    adj_normalized = normalize_adj(adj)
    laplacian = sp.eye(adj.shape[0]) - adj_normalized # L  = I  - A*
    largest_eigval, _ = eigsh(laplacian, 1, which='LM') # 最大特征值
    scaled_laplacian = (2. / largest_eigval[0]) * laplacian - sp.eye(adj.shape[0]) # 2/  *L - I

    t_k = list()
    t_k.append(sp.eye(adj.shape[0]))
    t_k.append(scaled_laplacian) # 切比雪夫多项式， 迭代添加

    def chebyshev_recurrence(t_k_minus_one, t_k_minus_two, scaled_lap):
        s_lap = sp.csr_matrix(scaled_lap, copy=True)
        return 2 * s_lap.dot(t_k_minus_one) - t_k_minus_two # 2 * T_{k1} - T_{k2}

    for i in range(2, k+1):
        t_k.append(chebyshev_recurrence(t_k[-1], t_k[-2], scaled_laplacian))

    return sparse_to_tuple(t_k)

def masked_softmax_cross_entropy(preds, labels, mask):
    """Softmax cross-entropy loss with masking."""
    loss = tf.nn.softmax_cross_entropy_with_logits(
        logits=preds,
        labels=labels)  # 对最后一层输出先做一个 softmax，之后对 softmax 的结果进行交叉熵计算
    mask = tf.cast(mask, dtype=tf.float32)  #  数据类型转变
    mask /= tf.reduce_mean(mask)  # 求平均值， reduce ：降维
    loss *= mask
    return tf.reduce_mean(loss)


def masked_accuracy(preds, labels, mask):
    """Accuracy with masking."""
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)


# inits 初始化的几种方法
def uniform(shape, scale=0.05, name=None):
    """Uniform init."""
    initial = tf.random_uniform(shape,
                                minval=-scale,
                                maxval=scale,
                                dtype=tf.float32)
    return tf.Variable(initial, name=name)


def glorot(shape, name=None):
    """Glorot & Bengio (AISTATS 2010) init."""
    init_range = np.sqrt(6.0 / (shape[0] + shape[1]))
    initial = tf.random_uniform(shape,
                                minval=-init_range,
                                maxval=init_range,
                                dtype=tf.float32)
    return tf.Variable(initial, name=name)


def zeros(shape, name=None):
    """All zeros."""
    initial = tf.zeros(shape, dtype=tf.float32)
    return tf.Variable(initial, name=name)


def ones(shape, name=None):
    """All ones."""
    initial = tf.ones(shape, dtype=tf.float32)
    return tf.Variable(initial, name=name)

_LAYER_UIDS = {}
def get_layer_uid(layer_name=''):
    """Helper function, assigns unique layer IDs."""
    if layer_name not in _LAYER_UIDS:
        _LAYER_UIDS[layer_name] = 1
        return 1
    else:
        _LAYER_UIDS[layer_name] += 1
        return _LAYER_UIDS[layer_name]


def sparse_dropout(x, keep_prob, noise_shape):
    """Dropout for sparse tensors."""
    random_tensor = keep_prob
    random_tensor += tf.random_uniform(noise_shape)  # [0,1] 之间 均匀分布
    dropout_mask = tf.cast(
        tf.floor(random_tensor),
        dtype=tf.bool)  # tf.floor 向下取整； （0.5， 1.5）得到 0，1 值， 转换为 boor 值
    pre_out = tf.sparse_retain(x, dropout_mask)  # 保留相应的 非空值
    return pre_out * (1. / keep_prob)


def dot(x, y, sparse=False):
    """Wrapper for tf.matmul (sparse vs dense)."""
    if sparse:
        res = tf.sparse_tensor_dense_matmul(x, y)  # * 稀疏矩阵相乘
    else:
        res = tf.matmul(x, y)  # 矩阵相乘
    return res


In [3]:
class Layer(object):
    """Base layer class. Defines basic API for all layer objects.
    Implementation inspired by keras (http://keras.io).

    # Properties
        name: String, defines the variable scope of the layer.
        logging: Boolean, switches Tensorflow histogram logging on/off

    # Methods
        _call(inputs): Defines computation graph of layer
            (i.e. takes input, returns output)
        __call__(inputs): Wrapper for _call() 
        _log_vars(): Log all variables
    """
    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            layer = self.__class__.__name__.lower()
            name = layer + '_' + str(get_layer_uid(layer))
        self.name = name
        self.vars = {}
        logging = kwargs.get('logging', False)
        self.logging = logging
        self.sparse_inputs = False

    def _call(self, inputs):
        return inputs

    def __call__(self, inputs):
        with tf.name_scope(self.name):  # 变量作用域
            if self.logging and not self.sparse_inputs:
                tf.summary.histogram(self.name + '/inputs', inputs)
            outputs = self._call(inputs)
            if self.logging:
                tf.summary.histogram(self.name + '/outputs', outputs)
            return outputs

    def _log_vars(self):
        for var in self.vars:
            tf.summary.histogram(self.name + '/vars/' + var, self.vars[var])


class Dense(Layer):
    """Dense layer."""
    def __init__(self,
                 input_dim,
                 output_dim,
                 placeholders,
                 dropout=0.,
                 sparse_inputs=False,
                 act=tf.nn.relu,
                 bias=False,
                 featureless=False,
                 **kwargs):
        super(Dense, self).__init__(**kwargs)

        if dropout:
            self.dropout = placeholders['dropout']
        else:
            self.dropout = 0.

        self.act = act  # 激活函数 relu
        self.sparse_inputs = sparse_inputs  # 输入的是否是稀疏矩阵
        self.featureless = featureless  # 节点是否有特征
        self.bias = bias

        # helper variable for sparse dropout
        self.num_features_nonzero = placeholders[
            'num_features_nonzero']  # 检查稀疏变量随机 dropout 是否有效。

        with tf.variable_scope(self.name + '_vars'):  # 变量作用域
            self.vars['weights'] = glorot([input_dim, output_dim],
                                          name='weights')
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')

        if self.logging:
            self._log_vars()

    def _call(self, inputs):
        x = inputs

        # dropout
        if self.sparse_inputs:
            x = sparse_dropout(x, 1 - self.dropout, self.num_features_nonzero)
        else:
            x = tf.nn.dropout(x, 1 - self.dropout)

        # transform
        output = dot(x, self.vars['weights'], sparse=self.sparse_inputs)

        # bias
        if self.bias:
            output += self.vars['bias']

        return self.act(output)


class GraphConvolution(Layer):
    """Graph convolution layer."""
    def __init__(self,
                 input_dim,
                 output_dim,
                 placeholders,
                 dropout=0.,
                 sparse_inputs=False,
                 act=tf.nn.relu,
                 bias=False,
                 featureless=False,
                 **kwargs):
        super(GraphConvolution, self).__init__(**kwargs)

        if dropout:
            self.dropout = placeholders['dropout']
        else:
            self.dropout = 0.

        self.act = act
        self.support = placeholders['support']
        self.sparse_inputs = sparse_inputs
        self.featureless = featureless
        self.bias = bias  # 是不是有 bias

        # helper variable for sparse dropout
        self.num_features_nonzero = placeholders['num_features_nonzero']

        with tf.variable_scope(self.name + '_vars'):
            for i in range(len(self.support)):
                self.vars['weights_' + str(i)] = glorot(
                    [input_dim, output_dim], name='weights_' + str(i))  # 初始化 权重
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')

        if self.logging:
            self._log_vars()

    def _call(self, inputs):
        x = inputs

        # dropout
        if self.sparse_inputs:
            x = sparse_dropout(x, 1 - self.dropout, self.num_features_nonzero) # 稀疏矩阵 随机 dropout
        else:
            x = tf.nn.dropout(x, 1 - self.dropout)

        # convolve
        supports = list()
        for i in range(len(self.support)):
            if not self.featureless:
                pre_sup = dot(x,
                              self.vars['weights_' + str(i)],
                              sparse=self.sparse_inputs)  # X * W
            else:
                pre_sup = self.vars['weights_' + str(i)]  # W
            support = dot(self.support[i], pre_sup, sparse=True)
            supports.append(support)
        output = tf.add_n(supports)

        # bias
        if self.bias:
            output += self.vars['bias']

        return self.act(output)

In [12]:
# Models
class Model(object):
    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            name = self.__class__.__name__.lower()
        self.name = name

        logging = kwargs.get('logging', False)
        self.logging = logging

        self.vars = {}
        self.placeholders = {}

        self.layers = []
        self.activations = []

        self.inputs = None
        self.outputs = None

        self.loss = 0
        self.accuracy = 0
        self.optimizer = None
        self.opt_op = None

    def _build(self):
        raise NotImplementedError

    def build(self):
        """ Wrapper for _build() """
        with tf.variable_scope(self.name):
            self._build()

        # Build sequential layer model
        self.activations.append(self.inputs)
        for layer in self.layers:
            hidden = layer(self.activations[-1])
            self.activations.append(hidden)
        self.outputs = self.activations[-1]

        # Store model variables for easy access
        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
        self.vars = {var.name: var for var in variables}

        # Build metrics
        self._loss()
        self._accuracy()

        self.opt_op = self.optimizer.minimize(self.loss)

    def predict(self):
        pass

    def _loss(self):
        raise NotImplementedError

    def _accuracy(self):
        raise NotImplementedError

    def save(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = saver.save(sess, "tmp/%s.ckpt" % self.name)
        print("Model saved in file: %s" % save_path)

    def load(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = "tmp/%s.ckpt" % self.name
        saver.restore(sess, save_path)
        print("Model restored from file: %s" % save_path)


class MLP(Model):
    def __init__(self, placeholders, input_dim, **kwargs):
        super(MLP, self).__init__(**kwargs)

        self.inputs = placeholders['features']
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders

        self.optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)

        self.build()

    def _loss(self):
        # Weight decay loss
        for var in self.layers[0].vars.values():
            self.loss += weight_decay * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'],
                                                  self.placeholders['labels_mask'])

    def _accuracy(self):
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels'],
                                        self.placeholders['labels_mask'])

    def _build(self):
        self.layers.append(Dense(input_dim=self.input_dim,
                                 output_dim=hidden1,
                                 placeholders=self.placeholders,
                                 act=tf.nn.relu,
                                 dropout=True,
                                 sparse_inputs=True,
                                 logging=self.logging))

        self.layers.append(Dense(input_dim=hidden1,
                                 output_dim=self.output_dim,
                                 placeholders=self.placeholders,
                                 act=lambda x: x,
                                 dropout=True,
                                 logging=self.logging))

    def predict(self):
        return tf.nn.softmax(self.outputs)


class GCN(Model):
    def __init__(self, placeholders, input_dim, **kwargs):
        super(GCN, self).__init__(**kwargs)

        self.inputs = placeholders['features']
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders

        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

        self.build()

    def _loss(self):
        # Weight decay loss
        for var in self.layers[0].vars.values():
            self.loss += weight_decay * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'],
                                                  self.placeholders['labels_mask'])

    def _accuracy(self):
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels'],
                                        self.placeholders['labels_mask'])

    def _build(self):

        self.layers.append(GraphConvolution(input_dim=self.input_dim,
                                            output_dim=hidden1,
                                            placeholders=self.placeholders,
                                            act=tf.nn.relu,
                                            dropout=True,
                                            sparse_inputs=True,
                                            logging=self.logging))

        self.layers.append(GraphConvolution(input_dim=hidden1,
                                            output_dim=self.output_dim,
                                            placeholders=self.placeholders,
                                            act=lambda x: x, #直接输出
                                            dropout=True,
                                            logging=self.logging))

    def predict(self):
        return tf.nn.softmax(self.outputs)

In [13]:
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

dataset_str  = "cora" # 数据集名称
model = "gcn" # 运行的模型 ： gcn，  mlp ， gcn_cheby
hidden1 = 16  # 'Number of units in hidden layer 1.'
learning_rate = 0.01 
weight_decay =  5e-4 # 'Weight for L2 loss on embedding matrix.'
epochs = 200
early_stopping = 10
dropout = 0.5
max_degree = 3  # 'Maximum Chebyshev polynomial degree.'

# Load data
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(dataset_str)

In [14]:
# Some preprocessing
features = preprocess_features(features)
if model == 'gcn':
    support = [preprocess_adj(adj)]
    num_supports = 1
    model_func = GCN
elif model == 'gcn_cheby':
    support = chebyshev_polynomials(adj, FLAGS.max_degree)
    num_supports = 1 + max_degree
    model_func = GCN
elif model == 'dense':
    support = [preprocess_adj(adj)]  # Not used
    num_supports = 1
    model_func = MLP
else:
    raise ValueError('Invalid argument for model: ' + str(model))
    

# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}

In [25]:
# Create model
model = model_func(placeholders, input_dim=features[2][1], logging=True)

# Initialize session
sess = tf.Session()


# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], (time.time() - t_test)


# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []

In [26]:
# Train model
for epoch in range(epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(features, support, y_train, train_mask, placeholders)
    feed_dict.update({placeholders['dropout']: dropout}) # 为什么 dropout 单独列出来

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)

    # Validation
    cost, acc, duration = evaluate(features, support, y_val, val_mask, placeholders)
    cost_val.append(cost)

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > early_stopping and cost_val[-1] > np.mean(cost_val[-(early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

# Testing
test_cost, test_acc, test_duration = evaluate(features, support, y_test, test_mask, placeholders)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))

Epoch: 0001 train_loss= 1.95458 train_acc= 0.10714 val_loss= 1.94950 val_acc= 0.39000 time= 0.06713
Epoch: 0002 train_loss= 1.94748 train_acc= 0.43571 val_loss= 1.94553 val_acc= 0.56800 time= 0.01894
Epoch: 0003 train_loss= 1.94132 train_acc= 0.67143 val_loss= 1.94109 val_acc= 0.64000 time= 0.01481
Epoch: 0004 train_loss= 1.93338 train_acc= 0.79286 val_loss= 1.93655 val_acc= 0.68400 time= 0.01547
Epoch: 0005 train_loss= 1.92743 train_acc= 0.75714 val_loss= 1.93228 val_acc= 0.71000 time= 0.01654
Epoch: 0006 train_loss= 1.91884 train_acc= 0.79286 val_loss= 1.92827 val_acc= 0.70400 time= 0.02040
Epoch: 0007 train_loss= 1.91248 train_acc= 0.82857 val_loss= 1.92439 val_acc= 0.70600 time= 0.01998
Epoch: 0008 train_loss= 1.90539 train_acc= 0.87857 val_loss= 1.92055 val_acc= 0.69800 time= 0.01909
Epoch: 0009 train_loss= 1.89513 train_acc= 0.82857 val_loss= 1.91658 val_acc= 0.69600 time= 0.02018
Epoch: 0010 train_loss= 1.89074 train_acc= 0.81429 val_loss= 1.91250 val_acc= 0.69600 time= 0.02083


Epoch: 0091 train_loss= 0.96822 train_acc= 0.95714 val_loss= 1.37101 val_acc= 0.78600 time= 0.03068
Epoch: 0092 train_loss= 0.96496 train_acc= 0.93571 val_loss= 1.36665 val_acc= 0.78400 time= 0.03010
Epoch: 0093 train_loss= 0.92467 train_acc= 0.93571 val_loss= 1.36190 val_acc= 0.78000 time= 0.02945
Epoch: 0094 train_loss= 0.93653 train_acc= 0.90714 val_loss= 1.35712 val_acc= 0.78000 time= 0.02081
Epoch: 0095 train_loss= 0.92579 train_acc= 0.94286 val_loss= 1.35189 val_acc= 0.78000 time= 0.01991
Epoch: 0096 train_loss= 0.87263 train_acc= 0.96429 val_loss= 1.34630 val_acc= 0.78200 time= 0.02149
Epoch: 0097 train_loss= 0.91380 train_acc= 0.95714 val_loss= 1.34096 val_acc= 0.78400 time= 0.02338
Epoch: 0098 train_loss= 0.93240 train_acc= 0.92143 val_loss= 1.33601 val_acc= 0.78600 time= 0.02202
Epoch: 0099 train_loss= 0.90114 train_acc= 0.94286 val_loss= 1.33095 val_acc= 0.78800 time= 0.01792
Epoch: 0100 train_loss= 0.92935 train_acc= 0.92857 val_loss= 1.32548 val_acc= 0.78800 time= 0.02026


Epoch: 0176 train_loss= 0.67686 train_acc= 0.97143 val_loss= 1.11638 val_acc= 0.78400 time= 0.02644
Epoch: 0177 train_loss= 0.62991 train_acc= 0.98571 val_loss= 1.11440 val_acc= 0.78200 time= 0.02898
Epoch: 0178 train_loss= 0.67007 train_acc= 0.92857 val_loss= 1.11259 val_acc= 0.78600 time= 0.02478
Epoch: 0179 train_loss= 0.64429 train_acc= 0.97143 val_loss= 1.11107 val_acc= 0.79000 time= 0.01974
Epoch: 0180 train_loss= 0.61495 train_acc= 0.95714 val_loss= 1.10932 val_acc= 0.78800 time= 0.02029
Epoch: 0181 train_loss= 0.60840 train_acc= 0.98571 val_loss= 1.10770 val_acc= 0.78800 time= 0.02288
Epoch: 0182 train_loss= 0.59963 train_acc= 0.97857 val_loss= 1.10590 val_acc= 0.78800 time= 0.02160
Epoch: 0183 train_loss= 0.66780 train_acc= 0.97143 val_loss= 1.10384 val_acc= 0.78800 time= 0.02194
Epoch: 0184 train_loss= 0.60548 train_acc= 0.97857 val_loss= 1.10159 val_acc= 0.78800 time= 0.02404
Epoch: 0185 train_loss= 0.66870 train_acc= 0.93571 val_loss= 1.09973 val_acc= 0.78800 time= 0.02245
